In [107]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib as plt

In [108]:
# Read CSV and create DataFrame
df = pd.read_csv("resources/purchase_data.csv")
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [109]:
# Explore data
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 7 columns):
Purchase ID    780 non-null int64
SN             780 non-null object
Age            780 non-null int64
Gender         780 non-null object
Item ID        780 non-null int64
Item Name      780 non-null object
Price          780 non-null float64
dtypes: float64(1), int64(3), object(3)
memory usage: 42.7+ KB


,Purchase ID,Age,Item ID,Price
count,780.000000,780.000000,780.000000,780.000000
mean,389.500000,22.714103,92.114103,3.050987
std,225.310896,6.659444,52.775943,1.169549
min,0.000000,7.000000,0.000000,1.000000
25%,194.750000,20.000000,48.000000,1.980000
50%,389.500000,22.000000,93.000000,3.150000
75%,584.250000,25.000000,139.000000,4.080000
max,779.000000,45.000000,183.000000,4.990000


In [110]:
## 1. Player Count

# Total number of players
total_players = len(df["SN"].value_counts())

player_count_df = pd.DataFrame({"Total Players":[total_players]})
player_count_df

,Total Players
0,576


In [111]:
## 2. Purchasing Analysis 

# Calculations
unique_items = len(df["Item Name"].value_counts())
avg_price = df["Price"].mean()
total_purchases = df["Purchase ID"].count()
total_revenue = df["Price"].sum()

# Create purchases dataframe
purchases_df = pd.DataFrame({"Number of Unique Items":[unique_items],
                             "Average Price":[avg_price],
                             "Number of Purchases":[total_purchases],
                             "Total Revenue":[total_revenue]})

# Format for currency
purchases_df = purchases_df.style.format({'Average Price':"${:,.2f}",
                           'Total Revenue': '${:,.2f}'})

purchases_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [112]:
## 3. Gender Demographics

# Group purchased_df by gender
gender_groups = df.groupby("Gender")

# Total genders by group
total_gender_count = gender_groups.nunique()["SN"]

percentage_of_players = (total_genders / total_players) * 100
percentage_of_players

gender_demographics_df = pd.DataFrame({"Percentage of Players":percentage_of_players,
                                       "Total Count":total_gender_count})

# Sort in descending order
gender_demographics_df.sort_values(["Total Count"], ascending = False).style.format({"Percentage of Players":"{:.1f}"})

,Percentage of Players,Total Count
Gender,,
Male,84.0,484
Female,14.1,81
Other / Non-Disclosed,1.9,11


In [114]:
## 4. Purchasing Analysis (Gender)

# Total purchases by gender
purchase_count = gender_groups["Purchase ID"].count()

# Average purchase price by gender
avg_purchase_price = gender_groups["Price"].mean()

# Average total purchases by gender
avg_purchase_total = gender_groups["Price"].sum()

# Average purchases 
# (SOMETHING GOING WRONG HERE!)
avg_purchases_per_person = avg_purchase_total / purchase_count

# Create dataframe
purchases_by_gender_df = pd.DataFrame({"Purchase Count": purchase_count,
                                    "Average Purchase Price":avg_purchase_price,
                                    "Average Purchase Value":avg_purchase_total,
                                    "Average Purchase Total per Person":avg_purchases_per_person})

# Format as currency
purchases_by_gender_df.style.format({"Average Purchase Value":"${:,.2f}",
                                  "Average Purchase Price":"${:,.2f}",
                                  "Avg Purchase Total per Person":"${:,.2f}"})

,Purchase Count,Average Purchase Price,Average Purchase Value,Average Purchase Total per Person
Gender,,,,
Female,113,$3.20,$361.94,0.0283452
Male,652,$3.02,"$1,967.64",0.00462861
Other / Non-Disclosed,15,$3.35,$50.19,0.223067


In [119]:
## 5. Age Demographics

# Establish bins for ages
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_names = ["< 10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Segment and sort age values into bins established above
df["Age Group"] = pd.cut(df["Age"],age_bins, labels=group_names)
df

# Create new data frame with the added "Age Group" and group it
age_grouped = df.groupby("Age Group")

# Count total players by age category
total_count_age = age_grouped["SN"].nunique()

# Calculate percentages by age category 
percentage_by_age = (total_count_age/total_players) * 100

# Create data frame with obtained values
age_demographics = pd.DataFrame({"Percentage of Players": percentage_by_age, "Total Count": total_count_age})

# Format the data frame with no index name in the corner
age_demographics.index.name = None

# Format percentage with two decimal places 
age_demographics.style.format({"Percentage of Players":"{:,.2f}"})

,Percentage of Players,Total Count
< 10,2.95,17
10-14,3.82,22
15-19,18.58,107
20-24,44.79,258
25-29,13.37,77
30-34,9.03,52
35-39,5.38,31
40+,2.08,12


In [121]:
## 6. Top Spenders

# Group purchase data by screen names
spender_stats = df.groupby("SN")

# Count the total purchases by name
purchase_count_spender = spender_stats["Purchase ID"].count()

# Calculate the average purchase by name 
avg_purchase_price_spender = spender_stats["Price"].mean()

# Calculate purchase total 
purchase_total_spender = spender_stats["Price"].sum()

# Create data frame with obtained values
top_spenders = pd.DataFrame({"Purchase Count": purchase_count_spender,
                             "Average Purchase Price": avg_purchase_price_spender,
                             "Total Purchase Value":purchase_total_spender})

# Sort in descending order to obtain top 5 spender names 
formatted_spenders = top_spenders.sort_values(["Total Purchase Value"], ascending=False).head()

# Format with currency style
formatted_spenders.style.format({"Average Purchase Total":"${:,.2f}",
                                 "Average Purchase Price":"${:,.2f}", 
                                 "Total Purchase Value":"${:,.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [123]:
# 7. Most Popular Items

# Create dataframe with item information
items = df[["Item ID", "Item Name", "Price"]]

# Group item data by ID and name
item_stats = items.groupby(["Item ID","Item Name"])

# Count the number purchases per item
purchase_count_item = item_stats["Price"].count()

# Calculate purchase value per item
purchase_value = (item_stats["Price"].sum()) 

# Find item price
item_price = purchase_value/purchase_count_item

# Create data frame
most_popular_items = pd.DataFrame({"Purchase Count": purchase_count_item, 
                                   "Item Price": item_price,
                                   "Total Purchase Value":purchase_value})

# Sort in descending order to for top 5 items
popular_formatted = most_popular_items.sort_values(["Purchase Count"], ascending=False).head()

# Format as currency
popular_formatted.style.format({"Item Price":"${:,.2f}",
                                "Total Purchase Value":"${:,.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [124]:
# 8. Most Profitable Items

# Highest purchase value
popular_formatted = most_popular_items.sort_values(["Total Purchase Value"],
                                                   ascending=False).head()
# Format as currency
popular_formatted.style.format({"Item Price":"${:,.2f}",
                                "Total Purchase Value":"${:,.2f}"})

popular_formatted

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
